### Problem 7 (6 points).

Numpy arrays may be used to some image processing. Here we try something cryptography-like.
Steganography, in general, is the practice to place some hidden data in the front layer of data.

#### 1) Here, we put some encoded string into an image. The goal of this task is to decrypt and print the message. 
Below, you can find some example of image processing. `redflower.bmp` is only in shades of red. If you zoom into `crypto.bmp` - top left corner - you can find some flaw in a picture. That is a place, where hidden message is encoded. To simplify decryption - all data is stored in one color layer. This is a text information. Encryption simply translates character (j=str[i]) into ASCII code (k=ord(j)). Therefore, decryption simply should find non-zero values in non-red layer and decrypt them from ASCII to string (chr(k)).

HINT: Here are the coordinates, if you don't want to search for non-zero data. To make the encoding effect more visible information is stored in the 8th-position in first dimension (table[7][][]). To guess last dimension solve => violet = red + ? (in RGB we have red,green,blue layers). Or simply print the numpy array:

```imtam = plt.imread('crypto.bmp')
       imtab
```       

#### 2) Write your own encription function. Could be the one described above.


In [69]:
from PIL import Image
import numpy as np
import scipy.misc
import random

# make red flower
def makered(picture_name, output_name):
    i = Image.open(picture_name)
    dim = np.zeros((i.size[1], i.size[0]), 'int8')   
    img_zero = Image.fromarray(dim, 'L')
    img = Image.merge('RGB', (i, img_zero, img_zero)) # in this variant will be easy to encode
    img.save(output_name) # (!) important to use bmp; jpg does some image conversion while saving
    return 
    
#change number from [0, rows*cols) to coordinates
def replace(n, rows, cols, cord):
    cord[0] = n // cols #coordinate of row
    cord[1] = n % cols #coordinate of col

#searching the first nonzero element in RGB-blue coordinates
def find_non_0(arr, rows, cols): #N = rows * cols
    first = np.zeros(2)
    n = 0
    end = 1
    while arr[int(first[0]), int(first[1]), 2] == 0 and n < rows * cols:
        n += 1
        replace(n, rows, cols, first)
    return n
        
#ENCODING
def encode(picture_name):
    img = Image.open(picture_name)
    img2arr = np.asarray(img, dtype=np.uint8)
    #to_encode = img2arr[7, 0:28, 2]
    #print(to_encode)
    nextp = np.zeros(2)
    rows = img2arr.shape[0]
    cols = img2arr.shape[1]
    start = find_non_0(img2arr, rows, cols)
    replace(start, rows, cols, nextp) #nextp denotes cordinates of first pixel to decoding
    while img2arr[int(nextp[0]), int(nextp[1]), 2] != 0 and start < rows * cols:
        replace(start, rows, cols, nextp)
        print(chr(img2arr[int(nextp[0]), int(nextp[1]), 2]), end='')
        start += 1
        replace(start, rows, cols, nextp)
        
#random number from [0, N-str_len]
def rand_num(N, str_len):
    if N < str_len:
        raise Exception("too long text to decoding")
    else:
        return(random.randint(0, N - str_len)) 

#DECODING text from a file
def decode(in_picture_name, out_picture_name, text_file_name):
    #text file to decoding:
    f = open(text_file_name, "r")
    text = f.read()
    f.close()
    #print(text)
    
    #picture to decoding:
    img = Image.open(in_picture_name)
    img2arr = np.asarray(img, dtype=np.uint8)
    
    #decoding
    img1arr = img2arr.copy()
    rows = img1arr.shape[0]
    cols = img1arr.shape[1]
    n = rand_num(rows * cols, len(text)) #beginning  
    
    for i in text:
        cord = np.zeros(2)
        replace(n, rows, cols, cord)
        img1arr[int(cord[0]), int(cord[1]), 2] = ord(i)
        n += 1
    
    img2 = Image.fromarray(img1arr)
    img2.save(out_picture_name)
    
    print("The picture is decoded.")
    

In [ ]:
# make red picture
makered('flower.jpg', 'redflower.bmp')

In [70]:
# encode the picture
encode('decoded.bmp') 

Congratulations !!! Got it! 

In [67]:
#decode the picture
decode('redflower.bmp', 'decoded.bmp', 'file.txt')

The picture is decoded.
